In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [3]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# df = pd.read_csv('dataset/training0_sort.csv')

C:\Users\samje\AppData\Local\Temp\ipykernel_7596\1228740871.py:1: DtypeWarning: Columns (68,69,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training0_sort.csv')


In [ ]:
# df.info()

In [ ]:
# display datatypes
# df.dtypes

In [ ]:
# df.dtypes.value_counts()

In [5]:
# run this
# df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)

In [38]:
df = pd.read_csv('dataset/training3_sort.csv')
df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)


C:\Users\samje\AppData\Local\Temp\ipykernel_7596\387851574.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training3_sort.csv')


In [7]:
# df

In [ ]:

# df.iloc[1,1:].values
# type(df.iloc[1,:].values)

In [25]:
# Our custom Dataset object
class RubiksDataset(Dataset):
    def __init__(self, df):
        self.df = df
        move_dict = {"#":0, "U":1, "U'":2, "U2":3, "D":4, "D'":5, "D2":6, "L":7, "L'":8, "L2":9, "R":10, 
                     "R'":11, "R2":12, "F":13, "F'":14, "F2":15, "B":16, "B'":17, "B2":18}
        # list of list, where inner list is an entire sequence.
        self.states = [] #is now a list of list of list :D
        self.moves = [] #list of list

        for i in range(len(df)):
            statelist = []
            movelist = []
            statemoves = df.iloc[i,1:]
            for line in statemoves:
                split = line.split(",")
                # state_as_int = int(split[0].replace(" ",""))/(10**54)
                state_as_str_list = split[0].split(" ")
                state_as_int_list = [int(i) for i in state_as_str_list]
                # above sets the input states to int values instead of string
                # below sets the string move into an int value,
                move_as_int = move_dict.get(split[1])

                statelist.append(state_as_int_list)
                movelist.append(move_as_int)
            self.states.append(statelist)
            self.moves.append(movelist)


        
    def __len__(self):
        return len(self.moves)
        
    def __getitem__(self, index):
        # inputs = torch.tensor(self.inputs[index]).float()
        # outputs = torch.tensor(self.outputs[index]).float()
        # print(self.states[index])
        inputs = torch.tensor(self.states[index]) # list of list, each index is a state vector
        outputs = torch.tensor(self.moves[index]) # list, each index is the move set of that sequence(index)
        return inputs, outputs

In [39]:
dataset = RubiksDataset(df)
# dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [ ]:
seqcount = 0
for inputs, outputs in dataloader:
    seqcount +=1
    print(outputs)
    # for i in outputs:
    #     print(i)
    print(seqcount)

In [8]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.layers = torch.nn.Sequential(torch.nn.Linear(108, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 128),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(128, 73))
        
    def forward(self, inputs, hidden):
        # combined = torch.tensor([inputs, hidden]).to(inputs.device)
        # print(inputs.shape)
        # print(hidden.shape)
        combined = torch.cat([inputs, hidden]).to(inputs.device)
        out = self.layers(combined)
        return out

In [9]:
model = RNN().to(device)
print(model.layers)

Sequential(
  (0): Linear(in_features=108, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=73, bias=True)
)


In [10]:
def train(model, dataloader, num_epochs, learning_rate, device):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    # hidden = torch.tensor([0]).to(device)
    # hidden= torch.zeros(1, 81, 54).to(device)
    for epoch in range(num_epochs):
        count = 0
        loss = 0

        
        # optimizer.zero_grad()
        for inputs, targets in dataloader:
            # inputs = inputs.permute(1,0,2)
            # need another loop inside.
            correct = 0
            pred = 0
            
            # reset hidden here
            loss = 0
            optimizer.zero_grad()
            # 
            for i in range(len(inputs)):
                hidden= torch.zeros(54).to(device)
                for j in range(len(inputs[i])):
                    outputs = model(inputs[i][j].to(device), hidden)
                    out, hidden = outputs[0:19], outputs[19::]
                    # print(out.shape)
                    # print(hidden.shape)
                    # print(targets[i][j].shape)

                    # print((torch.argmax(out) == targets[i][j]).item())
                    correct += (torch.argmax(out) == targets[i][j]).item()
                    pred += 1
                    loss += criterion(out.to(device), targets[i][j].to(device)) / len(inputs[i]) / len(inputs)
                # print(loss)
            batch_accuracy = correct/pred
            loss.backward()
            optimizer.step()
            count +=1
            print(f"Batch number: {count}")
            print(f"Batch {count} accuracy: {batch_accuracy}")
        
        # loss /= len(inputs)
        # loss.backward()
        # optimizer.step()

        #     outputs = model(inputs.to(device), hidden)
        #     out, hidden = outputs[0], outputs[1]
        #     loss += criterion(out.to(device), targets.to(device))
        # loss /= len(dataloader)
        # loss.backward()
        # optimizer.step()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Last Batch Accuracy: {batch_accuracy*100:.2f}%")

In [41]:
# Train the model
# model = RNN().to(device)
train(model, dataloader, num_epochs = 1, learning_rate = 0.0005, device = device)

Batch number: 1
Batch 1 accuracy: 0.7003012048192772
Batch number: 2
Batch 2 accuracy: 0.6601091867469879
Batch number: 3
Batch 3 accuracy: 0.686558734939759
Batch number: 4
Batch 4 accuracy: 0.6935240963855421
Batch number: 5
Batch 5 accuracy: 0.6785579819277109
Batch number: 6
Batch 6 accuracy: 0.6912650602409639
Batch number: 7
Batch 7 accuracy: 0.6926769578313253
Batch number: 8
Batch 8 accuracy: 0.6913591867469879
Batch number: 9
Batch 9 accuracy: 0.686558734939759
Batch number: 10
Batch 10 accuracy: 0.6947477409638554
Batch number: 11
Batch 11 accuracy: 0.6895707831325302
Batch number: 12
Batch 12 accuracy: 0.6980421686746988
Batch number: 13
Batch 13 accuracy: 0.6963478915662651
Batch number: 14
Batch 14 accuracy: 0.7030308734939759
Batch number: 15
Batch 15 accuracy: 0.6944653614457831
Batch number: 16
Batch 16 accuracy: 0.7008659638554217
Batch number: 17
Batch 17 accuracy: 0.697289156626506
Batch number: 18
Batch 18 accuracy: 0.7027484939759037
Batch number: 19
Batch 19 accur

In [42]:
# Save the model use new names each time tyvm
torch.save(model.state_dict(), 'weights/rnn_4layer_256_14.pth')

In [15]:
# Load the model
model = RNN().to(device)
model.load_state_dict(torch.load('weights/rnn_4layer_256_6.pth'))
# model.load_state_dict(torch.load('weights/rnn_4layer_256_6.pth', map_location=torch.device('cpu')))


<All keys matched successfully>

In [16]:
test_dataset = pd.read_csv("dataset/training99_sort.csv")
test_dataset.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
test_rubiksdataset = RubiksDataset(test_dataset)
test_dataloader = DataLoader(test_rubiksdataset, batch_size = 128)

C:\Users\samje\AppData\Local\Temp\ipykernel_7596\414264106.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  test_dataset = pd.read_csv("dataset/training99_sort.csv")


In [21]:
def test(model, test_loader, device):
    model.eval()
    # test_loss = 0
    correct = 0
    pred = 0
    count = 0
    sumofaccuracy = 0
    # criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            for i in range(len(data)):
                hidden= torch.zeros(54).to(device)
                for j in range(len(data[i])):
                    output = model(data[i][j].to(device), hidden)
                    out, hidden = output[0:19], output[19::]
                    # test_loss += criterion(out.to(device), target[i][j].to(device)) / len(data[i]) / len(data)
                    # pred = out.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                    # correct += pred.eq(target.view_as(pred)).sum().item()
                    # if args.dry_run:
                    #     break
                    correct += (torch.argmax(out) == target[i][j]).item()
                    pred += 1
            batch_accuracy = correct/pred

    # test_loss /= len(test_loader.dataset)

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     test_loss, correct, len(test_loader.dataset),
    #     100. * correct / len(test_loader.dataset)))
            count +=1
            print(f"Batch {count} accuracy: {batch_accuracy}")
            sumofaccuracy += batch_accuracy
    print(f"Total Test accuracy: {sumofaccuracy/count} , for {count} batch in total.")

In [43]:
test(model, test_dataloader, device=device)


Batch 1 accuracy: 0.7638578869047619
Batch 2 accuracy: 0.7575799851190477
Batch 3 accuracy: 0.7565724206349206
Batch 4 accuracy: 0.7555338541666666
Batch 5 accuracy: 0.7542410714285714
Batch 6 accuracy: 0.7521236359126984
Batch 7 accuracy: 0.7504517431972789
Batch 8 accuracy: 0.7488723028273809
Batch 9 accuracy: 0.7474371693121693
Batch 10 accuracy: 0.7462146577380953
Batch 11 accuracy: 0.745019954004329
Batch 12 accuracy: 0.7439778645833334
Batch 13 accuracy: 0.7430317078754579
Batch 14 accuracy: 0.7421077806122449
Batch 15 accuracy: 0.7412574404761905
Batch 16 accuracy: 0.7406877790178571
Batch 17 accuracy: 0.7399334733893558
Batch 18 accuracy: 0.7394128224206349
Batch 19 accuracy: 0.7385896381578947
Batch 20 accuracy: 0.7377464657738095
Batch 21 accuracy: 0.7371120323129252
Batch 22 accuracy: 0.7363492627164502
Batch 23 accuracy: 0.7357700892857143
Batch 24 accuracy: 0.7350725446428571
Batch 25 accuracy: 0.7343415178571429
Batch 26 accuracy: 0.7338205414377289
Batch 27 accuracy: 0.7

In [ ]:
# shawns stuff

In [ ]:
# split data into features
targets_numpy = train.label.values
features_numpy = train.loc[:,train.columns != "label"].values/255 # normalization

# train test split. Size of train data is 80% and size of test data is 20%. 
features_train, features_test, targets_train, targets_test = train_test_split(features_numpy,
                                                                             targets_numpy,
                                                                             test_size = 0.2,
                                                                             random_state = 42) 

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor) # data type is long

# batch_size, epoch and iteration
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

# visualize one of the images in data set
plt.imshow(features_numpy[10].reshape(28,28))
plt.axis("off")
plt.title(str(targets_numpy[10]))
plt.show()

In [5]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

# batch_size, epoch and iteration
batch_size = 100
n_iters = 8000
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_dim = 28    # input dimension
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 10   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

NameError: name 'features_train' is not defined

In [ ]:
# Training model
seq_dim = 28  
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        train  = Variable(images.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 250 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images.view(-1, seq_dim, input_dim))
                
                # Forward propagation
                outputs = model(images)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data[0], accuracy))

In [ ]:
# visualization loss 
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("RNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy 
plt.plot(iteration_list,accuracy_list,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("RNN: Accuracy vs Number of iteration")
plt.savefig('graph.png')
plt.show()